In [1]:
import json
import requests
from bs4 import BeautifulSoup
from datetime import datetime

In [2]:
def make_request(url):

    headers = {
        'authority': 'www.trulia.com',
        'accept': 'text/html,application/xhtml+xml,application/xml;q=0.9,image/avif,image/webp,image/apng,*/*;q=0.8,application/signed-exchange;v=b3;q=0.7',
        'accept-language': 'en-GB,en;q=0.9',
        'cache-control': 'max-age=0',
        # 'cookie': '_pxhd=Pmb3UA-RMlYLkHhAw786hzO-/FQ2JBydYx49PDsBYvMP6VDr6qK-84YmwcHgw6LWtkw98kFUQ-vNM52VuK-Ntw==:Z46o2iA3BbA2BqBqTSX5/HspXqyOcSkfMOcFb/A5Xv/Q8xlWGqrHwtNaGuPHn/EzoOWj8CCne84BaXbjN3f39Fslueg2sKWpOfUnwyTZyWQ=; _csrfSecret=m10N9JP%2B%2B5NTUsk0p71RqC4E; tlftmusr=240303s9rc2qbdq0cygoihipe4uum483; tabc=%7B%221274%22%3A%22a%22%2C%221337%22%3A%22b%22%2C%221341%22%3A%22b%22%2C%221353%22%3A%22b%22%2C%221365%22%3A%22b%22%2C%221377%22%3A%22b%22%2C%221386%22%3A%22b%22%2C%221395%22%3A%22c%22%2C%221406%22%3A%22a%22%2C%221409%22%3A%22b%22%2C%221425%22%3A%22a%22%2C%221437%22%3A%22a%22%2C%221439%22%3A%22d%22%2C%221440%22%3A%22b%22%2C%221444%22%3A%22a%22%2C%221452%22%3A%22a%22%2C%221464%22%3A%22a%22%2C%221469%22%3A%22a%22%2C%221475%22%3A%22b%22%2C%221476%22%3A%22control%22%2C%221478%22%3A%22control%22%7D; s_fid=2251A124A4CE106A-27CCB8FB6F01E30D; s_cc=true; s_vi=[CS]v1|32F207727EA2A373-6000102800046115[CE]; zjs_user_id=null; zg_anonymous_id=%2291cef6a3-d7c2-4219-98ca-32373d792a32%22; zjs_anonymous_id=%22240303s9rc2qbdq0cygoihipe4uum483%22; g_state=googleOneTap; pxcts=7e3bc907-d921-11ee-9a22-ed7bb3190c7b; _pxvid=7cb97b4c-d921-11ee-9a6e-600de7ee500f; _pxff_tm=1; trul_visitTimer=1709444837428_1709444912490; _px3=8868281e86f2827d707cc36ebac84624557fcdadc78035a20091a0953c205747:tU9xoq9VpH3dufWeoxum7F+D7OFM4wp3ldC0wRmg2S6alDGdf/tWzoHOsmmotOAIdSUKfgX8095Xafbd2XhGtQ==:1000:Zsb/PTDUG4Z3To4P61akFT9ianhi5HLC4/vdiLuvxd3sMBMA06k5yYFfCMAIv/ndQszki1NtEXZ3XS7TVJElzD84i+z1Ovq0i8Rsl0r+y3KHQrW4W2q506nIs93R/uSOBvbe4/ujmkO/wcWdYpoHCvD2aMTQyzsO/lyFXrVhLNQlyRPawEQ4ZL7Hs2mi6Q/kguposTdz6643AUbxCDaqSVW86QP5g5MlmhchPEPbs04=; s_sq=truliacom%3D%2526c.%2526a.%2526activitymap.%2526page%253Dbuy%25253Apdp%25253Aoverview%2526link%253DSkip%252520main%252520navigation%252520Buy%252520Rent%252520Mortgage%252520Saved%252520Homes%252520Saved%252520Searches%252520Sign%252520up%252520or%252520Log%252520in%252520Back%252520to%252520Search%252520For%252520Sale%252520NY%252520Albany%25252012210%252520372%252520%2526region%253DBODY%2526pageIDType%253D1%2526.activitymap%2526.a%2526.c%2526pid%253Dbuy%25253Apdp%25253Aoverview%2526pidt%253D1%2526oid%253Dfunctionrg%252528%252529%25257B%25257D%2526oidt%253D2%2526ot%253DDIV; _dd_s=rum=0&expire=1709445893634',
        'referer': 'https://www.trulia.com/',
        'sec-ch-ua': '"Not A(Brand";v="99", "Google Chrome";v="121", "Chromium";v="121"',
        'sec-ch-ua-mobile': '?0',
        'sec-ch-ua-platform': '"Windows"',
        'sec-fetch-dest': 'document',
        'sec-fetch-mode': 'navigate',
        'sec-fetch-site': 'same-origin',
        'sec-fetch-user': '?1',
        'upgrade-insecure-requests': '1',
        'user-agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/121.0.0.0 Safari/537.36',
    }
    try:
        response = requests.get(url, headers=headers)
        if response.status_code == 200:
            return response
    except requests.ConnectionError:
        pass
    except Exception as e:
        print(e.__class__)

In [3]:
def meta_data(pageProps):
    meta_data={}
    meta_data['totalProperties']=pageProps.get('totalProperties')
    meta_data['city']=pageProps["geo"].get("city")
    meta_data['state_code']=pageProps["geo"].get("state_code")
    meta_data['country']=pageProps["geo"].get("country")
    meta_data['slug_id']=pageProps["geo"].get("slug_id")

    return meta_data

In [4]:
def parse_page(response):
    soup=BeautifulSoup(response.content,"html.parser")
    print(soup.title.string)
    next_data_script = soup.find('script', {'id': '__NEXT_DATA__'})
    if next_data_script: 
        content = json.loads(next_data_script.string)
        props=content["props"]
        pageProps=props["pageProps"]
        properties=pageProps['properties']
        return properties
    else:
        print("Caotha Found")
        return None

In [5]:
def extract_property_details(data):
    details = {}

    details['property_id'] = data.get('property_id', '')
    details['list_price'] = data.get('list_price', '')
    details['status'] = data.get('status', '')
    details['listing_id'] = data.get('listing_id', '')

    primary_photo = data.get('primary_photo', {})
    if primary_photo is not None:
        details['primary_photo'] = primary_photo.get('href', '')
    else:
        details['primary_photo'] = ''

    description = data.get('description', {})
    details['description'] = {
        'beds': description.get('beds', '0'),
        'baths': description.get('baths_consolidated', ''),
        'sqft': description.get('sqft', ''),
        'lot_sqft': description.get('lot_sqft', ''),
        'type': description.get('type', ''),
        'sold_price': description.get('sold_price', ''),
        'sold_date': description.get('sold_date', '')
    }

    location = data.get('location', {})
    details['location'] = {
        'address': {
            'line': location.get('address', {}).get('line', ''),
            'postal_code': location.get('address', {}).get('postal_code', ''),
            'state': location.get('address', {}).get('state', ''),
            'state_code': location.get('address', {}).get('state_code', ''),
            'city': location.get('address', {}).get('city', ''),
        },
        'coordinate': {
            'lat': location.get('coordinate', {}).get('lat', ''),
            'lon': location.get('coordinate', {}).get('lon', '')
        },
        'county': {
            'name': location.get('county', {}).get('name', ''),
            'fips_code': location.get('county', {}).get('fips_code', '')
        }
    }


    flags = data.get('flags', {})
    details['flags'] = {
        'is_coming_soon': flags.get('is_coming_soon', False),
        'is_new_listing': flags.get('is_new_listing', False),
        'is_price_reduced': flags.get('is_price_reduced', False),
        'is_foreclosure': flags.get('is_foreclosure', False),
        'is_new_construction': flags.get('is_new_construction', False),
        'is_pending': flags.get('is_pending', False),
        'is_contingent': flags.get('is_contingent', False)
    }



    date_input = data.get('list_date', '')
    formatted_date = datetime.strptime(date_input, "%Y-%m-%dT%H:%M:%S.%fZ").strftime("%Y-%m-%d %H:%M:%S")
    details['list_date'] = formatted_date
    return details


In [6]:
def parse_properties(properties):
    processed_properties =[]
    for property in properties:
        try:
            processed=extract_property_details(property)
            processed_properties.append(processed)
        except Exception as e:
            print(f"Cannot extract details")
            raise e
    return processed_properties


In [7]:
response=make_request('https://www.realtor.com/realestateandhomes-search/Stockton_CA/pg-18')

In [8]:
response.content

b'<!DOCTYPE html><html lang="en"><head><meta charSet="utf-8"/><script>window.kxdl = { jy: \'\' }; window.rdcdl = window.kxdl;  window.rdcdl.globalholdout = { isEnabled: window?.__OPTIMIZELY_CACHE_ENTRIES__?.[\'EXP_GLOBAL_HOLDOUT-undefined-{}\']?.isEnabled }; !function(n,e,i){if(!n){n=n||{},window.permutive=n,n.q=[],n.config=i||{},n.config.apiKey=e,n.config.environment=n.config.environment||\'production\';for(var o=[\'addon\',\'identify\',\'track\',\'trigger\',\'query\',\'segment\',\'segments\',\'ready\',\'on\',\'once\',\'user\',\'consent\'],r=0;r<o.length;r++){var t=o[r];n[t]=function(e){return function(){var i=Array.prototype.slice.call(arguments,0);n.q.push({functionName:e,arguments:i})}}(t)}}}(window.permutive,\'2336e88c-2653-4e72-9d49-c562730fd85f\',{});(function(g,o){g[o]=g[o]||function(){(g[o][\'q\']=g[o][\'q\']||[]).push(arguments)},g[o][\'t\']=1*new Date})(window,\'_googCsa\');</script><script data-testid="dfp" src="https://securepubads.g.doubleclick.net/tag/js/gpt.js" async=""

In [9]:
properties=parse_page(response)

Stockton, CA Real Estate & Homes for Sale | realtor.com®


In [10]:
data=parse_properties(properties)

In [11]:
len(data)

35